In [ ]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode

import os
import shutil
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model

import cv2
import numpy as np

In [ ]:
# Code for Recording Voice Using Java Script

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec*1000))
  b = b64decode(s.split(',')[1])
  with open('/content/drive/MyDrive/Project/Testing/audio3.wav','wb') as f:
    f.write(b)
  return '/content/drive/MyDrive/Project/Testing/audio3.wav'  

In [ ]:
record()

In [ ]:
# Code for Converting my recording into Mel-Spectrogram

In [ ]:
n_fft = 2048
hop_length = 512
n_mels = 128

y, sr = librosa.load("/content/drive/MyDrive/Project/Testing/audio3.wav")
emotion, _ = librosa.effects.trim(y)
S = librosa.feature.melspectrogram(emotion, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
S_DB = librosa.power_to_db(S, ref=np.max)
librosa.display.specshow(S_DB, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel');
plt.colorbar(format='%+2.0f dB');
plt.savefig("/content/drive/MyDrive/Project/Testing/" + 'melspec3.png')
plt.clf()

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


<Figure size 432x288 with 0 Axes>

In [ ]:
# Loading the Model

In [ ]:
classifier = load_model('/content/drive/MyDrive/Project/Speech_Emotion1.h5')

In [ ]:
# Reading the input

In [ ]:
input_im = cv2.imread("/content/drive/MyDrive/Project/Testing/melspec3.png")

input_im = input_im[:253 , 55: 330]
input_im = cv2.resize(input_im, (150, 150), interpolation = cv2.INTER_LINEAR)

input_im = input_im.reshape(1,150,150,3) 

res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)

In [ ]:
# Checking res value against different emotions

In [ ]:
if res[0] == 0 :
    print("Angry")
elif res[0] == 1 :
    print("Disgust")
elif res[0] == 2 :
    print("Fear")
elif res[0] == 3 :
    print("Happy")
elif res[0] == 4 :
    print("Neutral")
elif res[0] == 5 :
    print("Sad")
elif res[0] == 6 :
    print("Surprise")   

Happy
